In [1]:
import numpy as np
import pandas as pd
import os
from kmeans import kmeans, avg_iou
from PIL import Image

In [2]:
datapath = "E:\\shangfangxin\\Project\\Xuelang\\train"
classes = os.listdir(datapath)

In [3]:
train_pd = pd.read_csv("abnormal.csv")
train_pd.head()

,filename,classes,xmin,ymin,xmax,ymax
0,J01_2018.06.16 10_24_16.jpg,修印,1345,1157,1761,1266
1,J01_2018.06.22 08_45_25.jpg,剪洞,1205,716,1351,792
2,J01_2018.06.23 09_09_55.jpg,剪洞,1206,1136,1564,1318
3,J01_2018.06.23 13_28_32.jpg,剪洞,460,466,518,538
4,J01_2018.06.26 09_55_27.jpg,剪洞,2200,1000,2262,1062


In [7]:
train_pd.describe()

,xmin,ymin,xmax,ymax,width,height,ratio
count,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000
mean,1150.279037,505.008499,1439.655807,1108.341360,289.376771,603.332861,2.185881
std,601.281766,413.692554,603.216957,510.617291,502.396080,637.242765,5.615563
min,1.000000,1.000000,94.000000,102.000000,31.000000,35.000000,0.026403
25%,733.000000,94.000000,1053.000000,721.750000,77.250000,104.000000,0.093373
50%,1205.500000,466.000000,1426.000000,1061.500000,113.000000,239.000000,0.697224
75%,1528.500000,805.250000,1860.250000,1552.250000,204.750000,990.000000,1.171993
max,2444.000000,1688.000000,2560.000000,1920.000000,2554.000000,1916.000000,72.685714


In [4]:
train_pd['width'] = train_pd['xmax'] - train_pd['xmin']
train_pd['height'] = train_pd['ymax'] - train_pd['ymin']
train_pd['ratio'] = train_pd['width'] / train_pd['height']

In [5]:
dataset = []
for _,row in train_pd.iterrows():
    tempImg = Image.open( os.path.join(datapath, row['classes'], row['filename']) )
    width, height = tempImg.size
    xmin = float(row['xmin'] / width)
    ymin = float(row['ymin'] / height)
    xmax = float(row['xmax'] / width)
    ymax = float(row['ymax'] / height)
    dataset.append([xmax - xmin, ymax - ymin])
dataset = np.array(dataset)
dataset.shape

(706, 2)

In [6]:
# width / height
out = kmeans(dataset, k = 3)
print("Accuracy: {:.2f}%".format(avg_iou(dataset, out) * 100))
print("Boxes:\n {}".format(out))

ratios = np.around(out[:, 0] / out[:, 1], decimals=2).tolist()
print("Ratios:\n {}".format(sorted(ratios)))

Accuracy: 48.56%
Boxes:
 [[0.0359375  0.70104167]
 [0.04121094 0.05729167]
 [0.22617187 0.090625  ]]
Ratios:
 [0.05, 0.72, 2.5]


In [8]:
candidate_boxes = [ (113, 113/r) for r in ratios ] + [ (239*r, 239) for r in ratios ]

In [26]:
candidate_boxes

[(113, 156.94444444444446),
 (113, 45.2),
 (11.950000000000001, 239),
 (172.07999999999998, 239),
 (597.5, 239)]

In [25]:
del(candidate_boxes[0])

In [12]:
normal_image_list = os.listdir( os.path.join(datapath, "正常") )
len(normal_image_list)

1316

In [27]:
cache = []
for file in normal_image_list:
    box = candidate_boxes[ np.random.choice(len(candidate_boxes), 1)[0] ]
    
    tempImg = Image.open( os.path.join(datapath, "正常", file) )
    width, height = tempImg.size
    
    # 裁剪图像中心位置
    xmin = max(int(width / 2 - box[0] / 2), 0)
    xmax = min(int(width / 2 + box[0] / 2), width)
    ymin = max(int(height / 2 - box[1] / 2), 0)
    ymax = min(int(height / 2 + box[1] / 2), height)
    cache.append( [file, "正常",  xmin, ymin, xmax, ymax ])
data_pd = pd.DataFrame(cache, columns=["filename", "classes", "xmin", "ymin", "xmax", "ymax"])

In [29]:
data_pd.head()

,filename,classes,xmin,ymin,xmax,ymax
0,J01_2018.06.13 13_23_08.jpg,正常,1223,881,1336,1038
1,J01_2018.06.13 13_24_39.jpg,正常,981,840,1578,1079
2,J01_2018.06.13 13_25_51.jpg,正常,981,840,1578,1079
3,J01_2018.06.13 13_27_54.jpg,正常,981,840,1578,1079
4,J01_2018.06.13 13_29_52.jpg,正常,1274,840,1285,1079


In [30]:
data_pd.to_csv("normal.csv", index=False)